<a href="https://colab.research.google.com/github/joe-jachim/cassava-leaf-classifier/blob/main/all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

from google.colab import drive
drive.mount('/content/drive')

<IPython.core.display.Javascript object>

In [23]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from tqdm import tqdm
import pdb

In [ ]:
# Define a more flexible model
class ResNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(28 * 28, 64)
    self.l2 = nn.Linear(64, 64)
    self.l3 = nn.Linear(64, 10)
    self.do = nn.Dropout(0.1)

  def forward(self, x):
    h1 = nn.functional.relu(self.l1(x))
    h2 = nn.functional.relu(self.l2(h1))
    do = self.do(h2 + h1)
    logits = self.l3(do)
    return logits

  #def backward(self, x)

model = ResNet().cuda()

In [20]:
# Define optimizer
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [21]:
# Define my loss
loss = nn.CrossEntropyLoss()

In [26]:
# Define  model
model = nn.Sequential(
    nn.Linear(3 * 600 * 800, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(64, 5)
).cuda()

In [12]:
def load_dataset():
    data_path = 'drive/MyDrive/data/cassava-leaf-disease-classification/train_images_labelled'
    train_dataset = datasets.ImageFolder(
        root=data_path,
        transform=transforms.ToTensor()
    )
    train_loader = DataLoader(
        train_dataset,
        batch_size=256,
        num_workers=0,
        shuffle=True
    )
    return train_loader

In [13]:
train_loader = load_dataset()

In [29]:
  losses = list()
  model.train()
  i = 1
  for batch in tqdm(train_loader):
    x, y = batch
    
    x = x.view(x.size(0), -1).cuda()

    # Forward
    l = model(x)  # l: logits

    # Compute the objective function
    J = loss(l, y.cuda())

    # Cleaning the gradient
    model.zero_grad()

    # Complete partial derivates of J wrt to params
    J.backward()

    # Step in the opposite direction of the gradient
    optimizer.step()

    print(J.item())

    losses.append(J.item())

    i = i + 1


  1%|          | 1/84 [01:54<2:38:24, 114.51s/it]

1.61210036277771



  2%|▏         | 2/84 [03:43<2:34:11, 112.82s/it]

1.6100589036941528



  4%|▎         | 3/84 [05:29<2:29:36, 110.82s/it]

1.6100642681121826


KeyboardInterrupt: ignored